In [ ]:
import os
import sqlite3
import pandas as pd
import numpy as np
import seaborn as sns
import itertools
import matplotlib.pyplot as plt
from time import time
from pathlib import Path
import warnings

from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier, RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn import linear_model
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.calibration import CalibratedClassifierCV
import xgboost as xgb

from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer
from sklearn.decomposition import PCA, FastICA
from sklearn.pipeline import Pipeline

from sklearn import tree
import graphviz

In [ ]:
pd.set_option('display.max_columns', 999)

if 'original_figure_figsize' not in locals():
    original_figure_figsize = plt.rcParams['figure.figsize']  # Default is [6.0, 4.0]

# If you want to change it use the below.
# plt.rcParams['figure.figsize'] = [15.0, 10.0]

In [ ]:
# Jupyter Notebook Configs
%matplotlib inline
from IPython.display import display

import warnings
warnings.filterwarnings('ignore')

In [ ]:
def is_kaggle():
    return 'KAGGLE_KERNEL_RUN_TYPE' in os.environ

def line(width=40):
    print('-' * width)
    
def data_info(df, sample_size=10):
    # print(df.info())
    display(df.describe(include='all'))
    display(df.sample(sample_size))

def print_null_columns(df):
    print('Columns with null values:\n', df.isnull().sum())
    
def plot_tree(model, estimator_index=0, feature_names=None, class_names=None):
    '''
    Plot model.
    Usage: lot_tree(model, feature_names=X_train.columns, class_names=['Win', 'Draw', 'Lose'])
    '''
    estimator = model.estimators_[estimator_index]
    dot_data = tree.export_graphviz(
        estimator, out_file=None, feature_names=feature_names,
        class_names=class_names, filled=True, rounded=True)
    graph = graphviz.Source(dot_data) 
    display(graph)

def plot_xgb(model):
    '''
    Plot XGBClassifier.
    Usage: plot_xgb(model)
    '''
    display(xgb.plot_importance(model))
    display(xgb.to_graphviz(model, rankdir='LR'))

In [ ]:
start = time()

db_path = Path('/kaggle/input/soccer/database.sqlite' if is_kaggle() else 'data/database.sqlite')
conn = sqlite3.connect(str(db_path))
n_jobs = 1

player_data = pd.read_sql('SELECT * FROM player;', conn)
player_stats_data = pd.read_sql('SELECT * FROM Player_Attributes;', conn)
team_data = pd.read_sql('SELECT * FROM Team;', conn)
match_data = pd.read_sql('SELECT * FROM Match;', conn)

ROWS = ["country_id", "league_id", "season", "stage", "date", "match_api_id", "home_team_api_id", 
        "away_team_api_id", "home_team_goal", "away_team_goal", "home_player_1", "home_player_2",
        "home_player_3", "home_player_4", "home_player_5", "home_player_6", "home_player_7", 
        "home_player_8", "home_player_9", "home_player_10", "home_player_11", "away_player_1",
        "away_player_2", "away_player_3", "away_player_4", "away_player_5", "away_player_6",
        "away_player_7", "away_player_8", "away_player_9", "away_player_10", "away_player_11"]

match_data.dropna(subset=ROWS, inplace=True)
match_data = match_data.tail(1500)

In [ ]:
data_info(player_stats_data)

In [ ]:
data_info(team_data)

In [ ]:
data_info(player_data)

In [ ]:
data_info(match_data)

In [ ]:
def get_match_label(match):
    '''Derives a label for a given match.'''
    home_goals = match['home_team_goal']
    away_goals = match['away_team_goal']

    label = pd.DataFrame()
    label.loc[0, 'match_api_id'] = match['match_api_id']

    if home_goals > away_goals:
        label.loc[0, 'label'] = 'Win'
    if home_goals == away_goals:
        label.loc[0, 'label'] = 'Draw'
    if home_goals < away_goals:
        label.loc[0, 'label'] = 'Defeat'

    return label.loc[0]

def get_fifa_stats(match, player_stats):
    '''Aggregate FIFA stats for a given match.'''
    match_id = match.match_api_id
    date = match['date']
    players = ['home_player_1', 'home_player_2', 'home_player_3', "home_player_4", "home_player_5",
               "home_player_6", "home_player_7", "home_player_8", "home_player_9", "home_player_10",
               "home_player_11", "away_player_1", "away_player_2", "away_player_3", "away_player_4",
               "away_player_5", "away_player_6", "away_player_7", "away_player_8", "away_player_9",
               "away_player_10", "away_player_11"]
    player_stats_new = pd.DataFrame()
    names = []

    for player in players:
        player_id = match[player]
        stats = player_stats[player_stats.player_api_id == player_id]
        current_stats = stats[stats.date < date].sort_values(by='date', ascending=False)[:1]

        if np.isnan(player_id):
            # NOTE: Defaulting to the first item?
            overall_rating = pd.Series(0)
        else:
            current_stats.reset_index(inplace=True, drop=True)
            overall_rating = pd.Series(current_stats.loc[0, 'overall_rating'])

        name = '{}_overall_rating'.format(player)
        names.append(name)

        player_stats_new = pd.concat([player_stats_new, overall_rating], axis=1)

    player_stats_new.columns = names
    player_stats_new['match_api_id'] = match_id

    player_stats_new.reset_index(inplace=True, drop=True)

    return player_stats_new.iloc[0]

def get_fifa_data(matches, player_stats, path=None, data_exists=False):
    '''Gets FIFA data for all matches.'''
    if data_exists:
        fifa_data = pd.read_pickle(path)
    else:
        print('Collecting FIFA data for each match...')
        start = time()
        fifa_data = matches.apply(lambda x: get_fifa_stats(x, player_stats), axis=1)
        end = time()
        print('FIFA data collected in {:.1f} minutes'.format((end - start)/60))

    return fifa_data

def get_overall_fifa_rankings(fifa, get_overall=False):
    '''Get overall FIFA rankings from FIFA data.'''
    temp_data = fifa

    if get_overall:
        data = temp_data.loc[:, fifa.columns.str.contains('overall_rating')]
        data.loc[:, 'match_api_id'] = temp_data.loc[:, 'match_api_id']
    else:
        cols = fifa.loc[:, fifa.columns.str.contains('date_stat')]
        temp_data = fifa.drop(cols.columns, axis=1)
        data = temp_data

    return data

def get_last_matches(matches, date, team, x=10):
    '''Get the last matches of a given team.'''
    team_matches = matches[
        (matches['home_team_api_id'] == team) | (matches['away_team_api_id'] == team)
    ]
    last_matches = team_matches[team_matches.date < date].sort_values(by='date', ascending=False).iloc[0:x, :]
    return last_matches

def get_last_matches_against_each_other(matches, date, home_team, away_team, x=10):
    '''Get the last x matches of two given teams.'''
    home_matches = matches[(matches['home_team_api_id'] == home_team) &
                           (matches['away_team_api_id'] == away_team)]
    away_matches = matches[(matches['home_team_api_id'] == away_team) &
                           (matches['away_team_api_id'] == home_team)]
    total_matches = pd.concat([home_matches, away_matches])

    min_x = min(x, total_matches.shape[0])
    matches_before = total_matches[total_matches.date < date]
    matches_sorted = matches_before.sort_values(by='date', ascending=False)
    return matches_sorted.iloc[0:min_x, :]

def get_goals(matches, team):
    '''Get the goals of a specific team frm a set of matches.'''
    home_goals = int(matches.home_team_goal[matches.home_team_api_id == team].sum())
    away_goals = int(matches.away_team_goal[matches.away_team_api_id == team].sum())

    total_goals = home_goals + away_goals
    return total_goals

def get_goals_conceded(matches, team):
    '''Get the goals conceded of a specific team from a set of matches.'''
    home_goals = int(matches.home_team_goal[matches.away_team_api_id == team].sum())
    away_goals = int(matches.away_team_goal[matches.home_team_api_id == team].sum())

    total_goals = home_goals + away_goals
    return total_goals

def get_wins(matches, team):
    '''Get the number of wins of a specific team from a set of matches.'''
    home_wins = int(matches.home_team_goal[
        (matches.home_team_api_id == team) &
        (matches.home_team_goal > matches.away_team_goal)
    ].count())
    away_wins = int(matches.away_team_goal[
        (matches.away_team_api_id == team) &
        (matches.away_team_goal > matches.home_team_goal)
    ].count())
    total_wins = home_wins + away_wins
    return total_wins

def get_match_features(match, matches, x=10):
    '''Create match specific features for a given match.'''
    date = match.date
    home_team = match.home_team_api_id
    away_team = match.away_team_api_id

    matches_home_team = get_last_matches(matches, date, home_team, x)
    matches_away_team = get_last_matches(matches, date, away_team, x)

    last_matches_against = get_last_matches_against_each_other(
        matches, date, home_team, away_team, x # =3
    )

    home_goals = get_goals(matches_home_team, home_team)
    away_goals = get_goals(matches_away_team, away_team)
    home_goals_conceded = get_goals_conceded(matches_home_team, home_team)
    away_goals_conceded = get_goals_conceded(matches_away_team, away_team)

    result = pd.DataFrame()
    result.loc[0, 'match_api_id'] = match.match_api_id
    result.loc[0, 'league_id'] = match.league_id
    result.loc[0, 'home_team_goals_difference'] = home_goals - home_goals_conceded
    result.loc[0, 'away_team_goals_difference'] = away_goals - away_goals_conceded
    result.loc[0, 'games_won_home_team'] = get_wins(matches_home_team, home_team)
    result.loc[0, 'games_won_away_team'] = get_wins(matches_away_team, away_team)
    result.loc[0, 'games_against_won'] = get_wins(last_matches_against, home_team)
    result.loc[0, 'games_against_lost'] = get_wins(last_matches_against, away_team)

    return result.loc[0]

def create_feables(matches, fifa, bookkeepers, get_overall=False, horizontal=True, x=10, verbose=True):
    '''Create and aggregate features and labels for all matches.'''

    fifa_stats = get_overall_fifa_rankings(fifa, get_overall)

    if verbose:
        print('Generating match features...')

    start = time()

    match_stats = matches.apply(lambda x: get_match_features(x, matches, x=10), axis=1)

    dummies = pd.get_dummies(match_stats['league_id']).rename(columns=lambda x: 'League_' + str(x))
    match_stats = pd.concat([match_stats, dummies], axis=1)
    match_stats.drop(['league_id'], inplace=True, axis=1)
    end = time()

    if verbose:
        print('Match features generated in {:.1f} minutes'.format((end - start) / 60))

    if verbose:
        print('Generating match labels...')

    start = time()
    labels = matches.apply(get_match_label, axis=1)
    end = time()

    if verbose:
        print('Match labels generated in {:.1f} minutes'.format((end - start) / 60))

    if verbose:
        print('Generating bookkeeper data...')

    start = time()
    bk_data = get_bookkeeper_data(matches, bookkeepers, horizontal=True)
    bk_data.loc[:, 'match_api_id'] = matches.loc[:, 'match_api_id']
    end = time()
    if verbose:
        print('Bookkeeper data generated in {:.1f} minutes'.format((end - start) / 60))

    features = pd.merge(match_stats, fifa_stats, on='match_api_id', how='left')
    features = pd.merge(features, bk_data, on='match_api_id', how='left')
    feables = pd.merge(features, labels, on='match_api_id', how='left')

    feables.dropna(inplace=True)

    return feables

def train_classifier(clf, dm_reduction, X_train, y_train, cv_sets, params, scorer,
                     jobs, use_grid_search=True, best_components=None, best_params=None):
    '''Fits a classifier to the training data.'''
    start = time()
    if use_grid_search:
        estimators = [('dm_reduce', dm_reduction), ('clf', clf)]
        pipeline = Pipeline(estimators)

        grid_obj = model_selection.GridSearchCV(pipeline, param_grid=params, scoring=scorer, cv=cv_sets, n_jobs=jobs)
        grid_obj.fit(X_train, y_train)
        best_pipe = grid_obj.best_estimator_
    else:
        estimators = [('dm_reduce', dm_reduction(n_components=best_components)), ('clf', clf(best_params))]
        pipeline = Pipeline(estimators)
        best_pipe = pipeline.fit(X_train, y_train)

    end = time()

    print('Trained {} in {:.1f} minutes'.format(clf.__class__.__name__, (end - start) / 60))

    return best_pipe

def predict_labels(clf, best_pipe, features, target):
    '''Make predictions using a fit classifier based on scorer.'''
    start = time()
    y_pred = clf.predict(best_pipe.named_steps['dm_reduce'].transform(features))
    end = time()

    print('Made predictions in {:.4f} seconds'.format(end - start))
    return accuracy_score(target.values, y_pred)

def train_calibrate_predict(clf, dm_reduction, X_train, y_train, X_calibrate, y_calibrate,
                            X_test, y_test, cv_sets, params, scorer, jobs, use_grid_search=True, **kwargs):
    '''Train and predict using a classifier based on scorer.'''
    print('----------{}----------'.format(clf.__class__.__name__))
    print('Training a {} with {}...'.format(clf.__class__.__name__,
                                            dm_reduction.__class__.__name__))
    best_pipe = train_classifier(clf, dm_reduction, X_train, y_train, cv_sets, params, scorer, jobs)

    print('Calibrating probabilities of classifier...')
    start = time()
    # 'isotonic' fails for XGBoost with "ValueError: Buffer dtype mismatch, expected 'float' but got 'double'"
    # https://github.com/scikit-learn/scikit-learn/pull/14902
    clf = CalibratedClassifierCV(best_pipe.named_steps['clf'], cv='prefit', method='sigmoid')
    clf.fit(best_pipe.named_steps['dm_reduce'].transform(X_calibrate), y_calibrate)
    end = time()
    print('Calibrated {} in {:.1f} minutes'.format(clf.__class__.__name__, (end - start) / 60))

    print('Score of {} for training set: {:.4f}.'.format(
        clf.__class__.__name__, predict_labels(clf, best_pipe, X_train, y_train)))
    print('Score of {} for test set: {:.4f}.'.format(
        clf.__class__.__name__, predict_labels(clf, best_pipe, X_test, y_test)))

    return clf, best_pipe.named_steps['dm_reduce'], predict_labels(
        clf, best_pipe, X_train, y_train), predict_labels(clf, best_pipe, X_test, y_test)

def convert_odds_to_prob(match_odds):
    '''Converts bookkeeper odds to probabilities.'''
    match_id = match_odds.loc[:, 'match_api_id']
    bookkeeper = match_odds.loc[:, 'bookkeeper']
    win_odd = match_odds.loc[:, 'Win']
    draw_odd = match_odds.loc[:, 'Draw']
    loss_odd = match_odds.loc[:, 'Defeat']

    win_prob = 1 / win_odd
    draw_prob = 1 / draw_odd
    loss_prob = 1 / loss_odd

    total_prob = win_prob + draw_prob + loss_prob

    probs = pd.DataFrame()
    probs.loc[:, 'match_api_id'] = match_id
    probs.loc[:, 'bookkeeper'] = bookkeeper
    probs.loc[:, 'Win'] = win_prob / total_prob
    probs.loc[:, 'Draw'] = draw_prob / total_prob
    probs.loc[:, 'Defeat'] = loss_prob / total_prob

    return probs

# bookkeepers: Bookkeeper tag
def get_bookkeeper_data(matches, bookkeepers, horizontal=True):
    '''Aggregates bookkeeper data for all matches and bookkeepers'''
    bk_data = pd.DataFrame()

    for bookkeeper in bookkeepers:
        temp_data = matches.loc[:, matches.columns.str.contains(bookkeeper)]
        temp_data.loc[:, 'bookkeeper'] = str(bookkeeper)
        temp_data.loc[:, 'match_api_id'] = matches.loc[:, 'match_api_id']

        cols = temp_data.columns.values
        cols[:3] = ['Win', 'Draw', 'Defeat']
        temp_data.columns = cols
        temp_data.loc[:, 'Win'] = pd.to_numeric(temp_data['Win'])
        temp_data.loc[:, 'Draw'] = pd.to_numeric(temp_data['Draw'])
        temp_data.loc[:, 'Defeat'] = pd.to_numeric(temp_data['Defeat'])

        if horizontal:
            temp_data = convert_odds_to_prob(temp_data)
            temp_data.drop('match_api_id', axis=1, inplace=True)
            temp_data.drop('bookkeeper', axis=1, inplace=True)

            win_name = bookkeeper + '_Win'
            draw_name = bookkeeper + '_Draw'
            defeat_name = bookkeeper + '_Defeat'
            temp_data.columns.values[:3] = [win_name, draw_name, defeat_name]

            bk_data = pd.concat([bk_data, temp_data], axis=1)
        else:
            bk_data = bk_data.append(temp_data, ignore_index=True)

    if horizontal:
        temp_data.loc[:, 'match_api_id'] = matches.loc[:, 'match_api_id']

    return bk_data

def get_bookkeeper_probs(matches, bookkeepers, horizontal=False):
    '''Get bookkeeper data and convert to probabilities for vertical aggregation.'''
    data = get_bookkeeper_data(matches, bookkeepers, horizontal=False)
    probs = convert_odds_to_prob(data)
    return probs

def plot_confusion_matrix(y_test, X_test, clf, dim_reduce, path, cmap=plt.cm.Blues, normalize=False):
    '''Plot confusion matrix for given classifier and data.'''
    labels = ['Win', 'Draw', 'Defeat']
    cm = confusion_matrix(y_test, clf.predict(dim_reduce.transform(X_test)), labels)

    if normalize:
        cm = cm.astype('float') / cm.sum()

    sns.set_style('whitegrid', {'axes.grid': False})
    fig = plt.figure(1)
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    title = 'Confusion matrix of a {} with {}'.format(
        best_clf.base_estimator.__class__.__name__,
        best_dm_reduce.__class__.__name__
    )
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(labels))
    plt.xticks(tick_marks, labels, rotation=45)
    plt.yticks(tick_marks, labels)
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, round(cm[i, j], 2),
                 horizontalalignment='center',
                 color='white' if cm[i, j] > thresh else 'black')
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()

    y_pred = clf.predict(dim_reduce.transform(X_test))
    print(classification_report(y_test, y_pred))

def explore_data(features, inputs, path):
    '''Explore data by plotting KDE graphs.'''
    fig = plt.figure(1)
    fig.subplots_adjust(bottom=-1, left=0.025, top=2, right=0.975)
    fig.set_figheight(15)
    fig.set_figwidth(10)

    i = 1
    for col in features.columns:
        sns.set_style('whitegrid')
        sns.set_context('paper', rc={'lines.linewidth': 1})
        plt.subplot(10, 5, i)
        j = i - 1

        sns.distplot(inputs[inputs['label'] == 'Win'].iloc[:, j], hist=False, label='Win')
        sns.distplot(inputs[inputs['label'] == 'Draw'].iloc[:, j], hist=False, label='Draw')
        sns.distplot(inputs[inputs['label'] == 'Defeat'].iloc[:, j], hist=False, label='Defeat')
        plt.legend()
        i = i + 1

    DefaultSize = fig.get_size_inches()
    fig.set_size_inches((DefaultSize[0] * 1.2, DefaultSize[1] * 1.2))
    plt.show()

    labels = inputs.loc[:, 'label']
    class_weights = labels.value_counts() / len(labels)
    print(class_weights)

    feature_details = features.describe().transpose()

    return feature_details

def find_best_classifier(classifiers, dm_reductions, scorer, X_t, y_t, X_c, y_c,
                         X_v, y_v, cv_sets, params, jobs):
    '''Tune all classifier and dimensionality reduction combinations to find best classifier.'''
    clfs_return = []
    dm_reduce_return = []
    train_scores = []
    test_scores = []

    for dm in dm_reductions:
        for clf in classifiers:
            clf, dm_reduce, train_score, test_score = train_calibrate_predict(
                clf=clf, dm_reduction=dm, X_train=X_t, y_train=y_t, X_calibrate=X_c,
                y_calibrate=y_c, X_test=X_v, y_test=y_v, cv_sets=cv_sets,
                params=params[clf], scorer=scorer, jobs=jobs, use_grid_search=True)

            clfs_return.append(clf)
            dm_reduce_return.append(dm_reduce)
            train_scores.append(train_score)
            test_scores.append(test_score)

        return clfs_return, dm_reduce_return, train_scores, test_scores

def plot_training_results(classifiers, dm_reductions, train_scores, test_scores, path):
    '''Plot results of classifier training.'''
    sns.set_style('whitegrid')
    sns.set_context('paper', font_scale=1, rc={'lines.linewidth': 1})
    ax = plt.subplot(111)
    w = 0.5
    x = np.arange(len(train_scores))
    ax.set_yticks(x + w)
    ax.legend ((train_scores[0], test_scores[0]), ('Train Scores', 'Test Socores'))
    names = []

    for i in range(0, len(classifiers)):
        clf = classifiers[i]
        clf_name = clf.base_estimator.__class__.__name__
        dm = dm_reductions[i]
        dm_name = dm.__class__.__name__

        name = '{} with {}'.format(clf_name, dm_name)
        names.append(name)

    ax.set_yticklabels((names))
    plt.xlim(0.4, 0.6)
    plt.barh(x, test_scores, color='b', alpha=0.6)
    plt.title('Test Data Accuracy Scores')
    fig = plt.figure(1)
    plt.show()

def plot_bookkeeper_cf_matrix(matches, bookkeepers, path, verbose=False, normalize=True):
    '''Plot confusion matrix of bookkeeper predictions.'''
    if verbose:
        print('Obtaining labels...')

    y_test_temp = matches.apply(get_match_label, axis=1)

    if verbose:
        print('Obtaining bookkeeper probabilities...')

    bookkeeper_probs = get_bookkeeper_probs(matches, bookkeepers)
    bookkeeper_probs.reset_index(inplace=True, drop=True)
    bookkeeper_probs.dropna(inplace=True)

    if verbose:
        print('Obtaining bookkeeper labels...')

    y_pred_temp = pd.DataFrame()
    y_pred_temp.loc[:, 'bk_label'] = bookkeeper_probs[['Win', 'Draw', 'Defeat']].idxmax(axis=1)
    y_pred_temp.loc[:, 'match_api_id'] = bookkeeper_probs.loc[:, 'match_api_id']

    if verbose:
        print('Plotting confusion matrix...')

    results = pd.merge(y_pred_temp, y_test_temp, on='match_api_id', how='left')
    y_test = results.loc[:, 'label']
    y_pred = results.loc[:, 'bk_label']

    labels = ['Win', 'Draw', 'Defeat']
    cm = confusion_matrix(y_test, y_pred, labels)

    if normalize:
        cm = cm.astype('float') / cm.sum()

    sns.set_style('whitegrid', {'axes.grid': False})
    fig = plt.figure(1)
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    title = 'Confusion matrix of Bookkeeper predictions!'
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(labels))
    plt.xticks(tick_marks, labels, rotation=45)
    plt.yticks(tick_marks, labels)
    thresh = cm.max() / 2.

    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, round(cm[i, j], 2),
                 horizontalalignment='center',
                 color='white' if cm[i, j] > thresh else 'black')

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()

    print(classification_report(y_test, y_pred))
    print('Bookkeeper score for test set: {:.4f}.'.format(accuracy_score(y_test, y_pred)))

In [ ]:
# Generate FIFA data
fifa_data = get_fifa_data(match_data, player_stats_data, data_exists=False)

In [ ]:
data_info(fifa_data)

In [ ]:
# Create features and labels.
bk_cols = ['B365', 'BW', 'IW', 'LB', 'PS', 'WH', 'SJ', 'VC', 'GB', 'BS']
bk_cols_selected = ['B365', 'BW']
feables = create_feables(match_data, fifa_data, bk_cols_selected, get_overall=True)
inputs = feables.drop('match_api_id', axis=1)

# Explore data and create visualization.
labels = inputs.loc[:, 'label']
features = inputs.drop('label', axis=1)

In [ ]:
data_info(features)

In [ ]:
feature_details = explore_data(features, inputs, db_path)

# Split the data into train, calibrate and test data sets.
X_train_calibrate, X_test, y_train_calibrate, y_test = train_test_split(
    features, labels, test_size=0.2, random_state=42, stratify=labels)
X_train, X_calibrate, y_train, y_calibrate = train_test_split(
    X_train_calibrate, y_train_calibrate, test_size=0.3, random_state=42,
    stratify=y_train_calibrate)

# Create cross validation data splits.
cv_sets = model_selection.StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=5)
cv_sets.get_n_splits(X_train, y_train)

In [ ]:
# Initialize models and parameters

# Initialize classifiers
RF_clf = RandomForestClassifier(n_estimators=200, random_state=1, class_weight='balanced')
AB_clf = AdaBoostClassifier(n_estimators=200, random_state=2)
GNB_clf = GaussianNB()
KNN_clf = KNeighborsClassifier()
LOG_clf = linear_model.LogisticRegression(multi_class='ovr', solver='sag', class_weight='balanced')
XG_clf = xgb.XGBClassifier(n_estimators=200, random_state=3)
clfs = [RF_clf, AB_clf, GNB_clf, KNN_clf, LOG_clf, XG_clf]

# Grid search parameters
feature_len = features.shape[1]
scorer = make_scorer(accuracy_score)
dm_comp_count = np.arange(5, feature_len, np.around(feature_len / 5)).astype('int')
parameters_RF = {'clf__max_features': ['auto', 'log2'],
                 'dm_reduce__n_components': dm_comp_count}
parameters_AB = {'clf__learning_rate': np.linspace(0.5, 2, 5),
                 'dm_reduce__n_components': dm_comp_count}
parameters_GNB = {'dm_reduce__n_components': dm_comp_count}
parameters_KNN = {'clf__n_neighbors': [3, 5, 10],
                  'dm_reduce__n_components': dm_comp_count}
parameters_LOG = {'clf__C': np.logspace(1, 1000, 5),
                  'dm_reduce__n_components': dm_comp_count}
parameters_XG = {'clf__max_depth': [10, 50, 100],
                 'dm_reduce__n_components': dm_comp_count}
parameters = {clfs[0]: parameters_RF,
              clfs[1]: parameters_AB,
              clfs[2]: parameters_GNB,
              clfs[3]: parameters_KNN,
              clfs[4]: parameters_LOG,
              clfs[5]: parameters_XG}

# Dimensionality reduction
pca = PCA()
dm_reductions = [pca]

In [ ]:
# Train a baseline model (GBC)

clf = LOG_clf
clf.fit(X_train, y_train)
print('Score of {} for training set: {:.4f}.'.format(
    clf.__class__.__name__, accuracy_score(y_train, clf.predict(X_train))))
print('Score of {} for test set: {:.4f}.'.format(
    clf.__class__.__name__, accuracy_score(y_test, clf.predict(X_test))))

In [ ]:
# Find the best classifier

clfs, dm_reductions, train_scores, test_scores = find_best_classifier(
    clfs, dm_reductions, scorer, X_train, y_train, X_calibrate, y_calibrate,
    X_test, y_test, cv_sets, parameters, n_jobs)

plot_training_results(clfs, dm_reductions, np.array(train_scores),
                      np.array(test_scores), path=db_path)

In [ ]:
# Plot a confusion matrix of the best model and the bookkeeper predictions

best_clf = clfs[np.argmax(test_scores)]
best_dm_reduce = dm_reductions[np.argmax(test_scores)]
print('The best classifier is a {} with {}.'.format(
    best_clf.base_estimator.__class__.__name__, best_dm_reduce.__class__.__name__))
plot_confusion_matrix(y_test, X_test, best_clf, best_dm_reduce, path=db_path, normalize=True)

plot_bookkeeper_cf_matrix(match_data, bk_cols, db_path, verbose=True, normalize=True)

In [ ]:
end = time()
print('Program run in {:.1f} minutes'.format((end - start) / 60))